## ZeroRel data inspection

In [61]:
import json
with open('data/zero_rel_all.jsonl', 'r') as f:
    data = []
    for i in range(5):
        data.append(json.loads(next(f)))
    # data = [json.loads(line) for line in f]

In [62]:
i = 4

In [63]:
data[i].keys()

dict_keys(['ner', 'relations', 'tokenized_text'])

In [64]:
data[i]['ner'][:5]

[[0, 1, 'CARDINAL', 'Two'],
 [1, 6, 'ORG', 'Temple University School of Medicine'],
 [11, 12, 'ORG', 'NIH'],
 [15, 16, 'GPE', 'Philadelphia'],
 [17, 18, 'GPE', 'PA']]

In [65]:
len(data[i]['ner'])

24

In [66]:
" ".join(data[i]['tokenized_text'])

'Two Temple University School of Medicine Faculty Members Selected for Prestigious NIH Posts \n ( Philadelphia , PA ) – Joseph Cheung , MD , PhD , Senior Associate Dean for Research and Professor of Medicine at Temple University School of Medicine , and Xiao - Feng Yang , MD , PhD , Professor of Pharmacology , Professor at the Cardiovascular Research Center and Professor at the Sol Sherry Thrombosis Research Center at Temple University School of Medicine , have been selected for prestigious posts as members of National Institutes of Health ( NIH ) study sections . \n Members of NIH study sections are seasoned experts in their scientific disciplines , and selected on the basis of their research accomplishments , publication records and scientific honors . Study section members review grant applications submitted to the NIH from investigators all over the country , make recommendations on the applications to national advisory councils and survey the status of research in their fields . \

In [67]:
[r for r in data[i]['relations'] if r['relation_text'] == 'no_relation'][-3:]

[{'head': {'mention': 'Xiao-Feng Yang',
   'position': [44, 48],
   'type': 'PERSON'},
  'tail': {'mention': 'Joseph Cheung', 'position': [20, 22], 'type': 'PERSON'},
  'relation_text': 'no_relation'},
 {'head': {'mention': 'Xiao-Feng Yang',
   'position': [44, 48],
   'type': 'PERSON'},
  'tail': {'mention': 'Cheung', 'position': [166, 167], 'type': 'PERSON'},
  'relation_text': 'no_relation'}]

In [68]:
# get exceptions not considered for relations
# should include self-pairs 

seen_rels = set()

for rel in data[i]['relations']:
    seen_rels.add(((rel['head']['position'][0], rel['head']['position'][1]), (rel['tail']['position'][0], rel['tail']['position'][1])))

exception = []
for ent1 in data[i]['ner']:
    for ent2 in data[i]['ner']:

        if ((ent1[0], ent1[1]), (ent2[0], ent2[1])) not in seen_rels:
            exception.append((ent1, ent2))

In [69]:
exception[:4]

[([0, 1, 'CARDINAL', 'Two'], [0, 1, 'CARDINAL', 'Two']),
 ([1, 6, 'ORG', 'Temple University School of Medicine'],
  [1, 6, 'ORG', 'Temple University School of Medicine']),
 ([11, 12, 'ORG', 'NIH'], [11, 12, 'ORG', 'NIH']),
 ([15, 16, 'GPE', 'Philadelphia'], [15, 16, 'GPE', 'Philadelphia'])]

In [70]:
len(exception)

24

In [71]:
should_be_rels = (len(data[i]['ner']) * (len(data[i]['ner'])))
print(f"{should_be_rels} entity-entity pairs")

print(f"So should be {should_be_rels} (entity pairs) - {len(exception)} (self-pairs) => {len(data[i]['relations'])}")

576 entity-entity pairs
So should be 576 (entity pairs) - 24 (self-pairs) => 552
